#### SET DIRECTORIES !!!

In [1]:
data_directory="data"
directory="/Users/amaiasagastimartinez/Desktop/SENS-Soundlights/code/sens-sensor"#path/to/sens-sensor"
print(f"Data directory is {data_directory}")
print(f"Working directory is {directory}")

Data directory is data
Working directory is /Users/amaiasagastimartinez/Desktop/SENS-Soundlights/code/sens-sensor


# Code to prepare URBANSOUDSCAPES8K original dataset

This dataset (https://urbansounddataset.weebly.com/urbansound8k.html) contains 8732 labeled sound excerpts (<=4s) of urban sounds from 10 classes: air_conditioner, car_horn, children_playing, dog_bark, drilling, enginge_idling, gun_shot, jackhammer, siren, and street_music. The classes are drawn from the urban sound taxonomy.

For this project, we are training binary models for each sound class, therefore, in this script we are adding new columns to UrbanSound8K.csv indicating which sound source is present with '1'.


## 1) Analyse class and fold data balance

Count how many audios there are per sound class in each fold.

In [17]:
import pandas as pd
import numpy as np
import os

# Set the working directory
import os
os.chdir(directory)

# Imports from this project
from development.lib.auxiliars import US8k_sources

Paths, inputs

In [9]:
dataset_path=os.path.join(data_directory,"UrbanSoundscapes8K/metadata/UrbanSound8K.csv")
saving_path=os.path.join(data_directory,"files/UrbanSound8K_adapted.csv")

Little analysis of number of sound sources per fold

In [15]:
# Array to keep count, each position --> counts of class(columns) per fold(rows) 
count=np.zeros([10,10])
classes=["air_conditioner", "car_horn","children_playing","dog_bark","drilling", "engine_idling","gun_shot","jackhammer","siren","street_music"]
# Import csv file as dataframe
df = pd.read_csv(dataset_path)
for row in df.index:
    count[df["fold"][row]-1][df["classID"][row]]=count[df["fold"][row]-1][df["classID"][row]]+1

print("Rows are folds and columns are classes:")
print(count)
print("\n")
for fold_index in range(count.shape[0]):
    print("Fold ", fold_index+1, " total: ", count[fold_index,:].sum())
print("\n")
for class_index in range(count.shape[1]):
    print("Class ", classes[class_index], " total: ", count[:,class_index].sum())


Rows are folds and columns are classes:
[[100.  36. 100. 100. 100.  96.  35. 120.  86. 100.]
 [100.  42. 100. 100. 100. 100.  35. 120.  91. 100.]
 [100.  43. 100. 100. 100. 107.  36. 120. 119. 100.]
 [100.  59. 100. 100. 100. 107.  38. 120. 166. 100.]
 [100.  98. 100. 100. 100. 107.  40. 120.  71. 100.]
 [100.  28. 100. 100. 100. 107.  46.  68.  74. 100.]
 [100.  28. 100. 100. 100. 106.  51.  76.  77. 100.]
 [100.  30. 100. 100. 100.  88.  30.  78.  80. 100.]
 [100.  32. 100. 100. 100.  89.  31.  82.  82. 100.]
 [100.  33. 100. 100. 100.  93.  32.  96.  83. 100.]]


Fold  1  total:  873.0
Fold  2  total:  888.0
Fold  3  total:  925.0
Fold  4  total:  990.0
Fold  5  total:  936.0
Fold  6  total:  823.0
Fold  7  total:  838.0
Fold  8  total:  806.0
Fold  9  total:  816.0
Fold  10  total:  837.0


Class  air_conditioner  total:  1000.0
Class  car_horn  total:  429.0
Class  children_playing  total:  1000.0
Class  dog_bark  total:  1000.0
Class  drilling  total:  1000.0
Class  engine_idling

## 2) Add one column per sound source with 1 or 0 depeding on its presence in the audio


In [25]:
# Creating an empty DataFrame with the same columns as df plus the 'audio_path' + sources columns
new_columns = df.columns.tolist() + ['audio_path'] + US8k_sources
new_df = pd.DataFrame(columns=new_columns)

# Iterate over rows and apply the condition
for index, row in df.iterrows():
    # Generate the audio path
    audio_path = "UrbanSoundscapes8K/audio/fold"+str(row["fold"])+"/"+row["slice_file_name"]
    # Generate binary vector of sources
    sources_vector=np.zeros(len(US8k_sources))
    sources_vector[row["classID"]]=1
    # Append the row to the new DataFrame with the audio_path
    new_row = row.tolist() + [audio_path]+sources_vector.tolist()
    new_df.loc[len(new_df)] = new_row

# Reset index if needed
new_df.reset_index(drop=True, inplace=True)

print(new_df)


         slice_file_name    fsID       start         end  salience  fold  \
0       100032-3-0-0.wav  100032    0.000000    0.317551         1     5   
1     100263-2-0-117.wav  100263   58.500000   62.500000         1     5   
2     100263-2-0-121.wav  100263   60.500000   64.500000         1     5   
3     100263-2-0-126.wav  100263   63.000000   67.000000         1     5   
4     100263-2-0-137.wav  100263   68.500000   72.500000         1     5   
...                  ...     ...         ...         ...       ...   ...   
8727     99812-1-2-0.wav   99812  159.522205  163.522205         2     7   
8728     99812-1-3-0.wav   99812  181.142431  183.284976         2     7   
8729     99812-1-4-0.wav   99812  242.691902  246.197885         2     7   
8730     99812-1-5-0.wav   99812  253.209850  255.741948         2     7   
8731     99812-1-6-0.wav   99812  332.289233  334.821332         2     7   

      classID             class  \
0           3          dog_bark   
1           2  ch

Save in saving directory path

In [26]:
new_df.to_csv(saving_path, index=False)